## Analyze whether SNWD varies more from year to year or from place to place.

In [5]:
import pandas as pd
import numpy as np
import urllib
import math

In [10]:
import findspark
findspark.init()

from pyspark import SparkContext
#sc.stop()
sc = SparkContext(master="local[3]",pyFiles=['lib/numpy_pack.py','lib/spark_PCA.py','lib/computeStats.py'])

from pyspark import SparkContext
from pyspark.sql import *
sqlContext = SQLContext(sc)

In [11]:
import sys
sys.path.append('./lib')

import numpy as np
from numpy_pack import packArray,unpackArray
from spark_PCA import computeCov
from computeStats import computeOverAllDist, STAT_Descriptions

In [32]:
### Read the data frame from pickle file

data_dir='../../Data/Weather'
file_index='SBSSBBBS'
meas='SNWD'

from pickle import load

#read statistics
filename=data_dir+'/STAT_%s.pickle'%file_index
STAT,STAT_Descriptions = load(open(filename,'rb'))
print('keys from STAT=',STAT.keys())

#!ls -ld $data_dir/*.parquet

#read data
filename=data_dir+'/decon_%s_%s.parquet'%(file_index,meas)

df=sqlContext.read.parquet(filename)
print(df.count())
df.show(2)

 ('keys from STAT=', ['TMIN', 'TOBS', 'TMAX', 'SNOW', 'SNWD', 'PRCP'])
502
+------------------+-------------------+-----------------+---------+--------+--------+---------+-----------+-------------------+--------------------+-------------------+------------------+-----------+------------+------+--------------------+------+
|           coeff_1|            coeff_2|          coeff_3|elevation|   label|latitude|longitude|measurement|              res_1|               res_2|              res_3|          res_mean|    station|   total_var|undefs|              vector|  year|
+------------------+-------------------+-----------------+---------+--------+--------+---------+-----------+-------------------+--------------------+-------------------+------------------+-----------+------------+------+--------------------+------+
|-8864.636346120038|-2039.9548861316823|698.3849072461769|   1797.4|SBSSBBBS|  43.465|-113.5581|       SNWD| 0.0848993837998154| 0.03534173056103785|0.03086802364758516|0.7489632

In [33]:
print df.columns

['coeff_1', 'coeff_2', 'coeff_3', 'elevation', 'label', 'latitude', 'longitude', 'measurement', 'res_1', 'res_2', 'res_3', 'res_mean', 'station', 'total_var', 'undefs', 'vector', 'year']


In [59]:
#extract longitude and latitude for each station
feature='coeff_1'
sqlContext.registerDataFrameAsTable(df,'weather')
#Features=', '.join(['coeff_1', 'coeff_2', 'coeff_3', 'elevation', 'latitude', 'longitude',\
#          'res_1', 'res_2', 'res_3', 'res_mean', 'year'])
Features='station, year, coeff_3'
Query="SELECT %s FROM weather"%Features
print(Query)
pdf = sqlContext.sql(Query).toPandas()
pdf.head()

SELECT station, year, coeff_3 FROM weather


,station,year,coeff_3
0,USC00102260,1993.0,698.384907
1,USC00104845,1997.0,270.230987
2,USC00104845,1980.0,302.271603
3,USC00104230,1972.0,-1557.231484
4,USC00103942,1952.0,205.481422


In [60]:
year_station_table=pdf.pivot(index='year', columns='station', values='coeff_3')
year_station_table.head(10)

station,USC00100010,USC00100227,USC00100375,USC00100915,USC00102260,USC00102676,USC00102823,USC00103110,USC00103297,USC00103825,...,USC00108380,USC00108535,USC00108906,USC00108937,USC00109601,USC00480124,USW00024142,USW00024145,USW00024156,USW00094143
year,,,,,,,,,,,,,,,,,,,,,
1911.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1949.0,-1012.988804,-1400.150199,NaN,-1062.199464,NaN,NaN,-275.958091,NaN,-318.672393,NaN,...,-2113.391426,NaN,-126.976296,NaN,NaN,NaN,-2460.992271,-1541.666138,-861.381942,NaN
1950.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1769.028879,...,-796.176164,NaN,1223.904849,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1951.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1606.577544,...,-41.679069,NaN,980.014270,NaN,NaN,NaN,38.875430,NaN,NaN,NaN
1952.0,NaN,NaN,NaN,-141.167657,NaN,NaN,NaN,NaN,NaN,1362.970202,...,-768.137068,NaN,1667.050715,NaN,NaN,NaN,-469.183992,158.513209,-22.130661,NaN
1953.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,616.896595,...,NaN,NaN,-902.430275,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1954.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1053.118713,...,NaN,NaN,233.919976,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1955.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-207.072284,NaN,410.009012,NaN,NaN,NaN,NaN,-359.055411,NaN,NaN
1956.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,888.963159,NaN,NaN,NaN,NaN,-101.770253,NaN,NaN


### Estimating the effect of the year vs the effect of the station

To estimate the effect of time vs. location on the first eigenvector coefficient we
compute:

* The average row: `mean-by-station`
* The average column: `mean-by-year`

We then compute the RMS before and after subtracting either  the row or the column vector.

In [61]:
def RMS(Mat):
    return np.sqrt(np.nanmean(Mat**2))

mean_by_year=np.nanmean(year_station_table,axis=1)
mean_by_station=np.nanmean(year_station_table,axis=0)
tbl_minus_year = (year_station_table.transpose()-mean_by_year).transpose()
tbl_minus_station = year_station_table-mean_by_station

print 'total RMS                   = ',round(RMS(year_station_table),2) 
print 'RMS removing mean-by-station= ',round(RMS(tbl_minus_station),2), ',fraction explained= ',round(((RMS(year_station_table)-RMS(tbl_minus_station))/(RMS(year_station_table)))*100,2)
print 'RMS removing mean-by-year   = ',round(RMS(tbl_minus_year),2), ',fraction explained= ',round(((RMS(year_station_table)-RMS(tbl_minus_year))/(RMS(year_station_table)))*100,2)


total RMS                   =  784.22
RMS removing mean-by-station=  670.92 ,fraction explained=  14.45
RMS removing mean-by-year   =  588.68 ,fraction explained=  24.93


In [15]:
T=year_station_table
print 'initial RMS=',RMS(T)
for i in range(5):
    mean_by_year=np.nanmean(T,axis=1)
    T=(T.transpose()-mean_by_year).transpose()
    print i,'after removing mean by year    =',RMS(T)
    mean_by_station=np.nanmean(T,axis=0)
    T=T-mean_by_station
    print i,'after removing mean by stations=',RMS(T)

initial RMS= 848.69917793
0 after removing mean by year    = 540.693118362
0 after removing mean by stations= 526.181161298
1 after removing mean by year    = 524.635943663
1 after removing mean by stations= 524.321748828
2 after removing mean by year    = 524.224364916
2 after removing mean by stations= 524.186093351
3 after removing mean by year    = 524.169321724
3 after removing mean by stations= 524.161576845
4 after removing mean by year    = 524.157893966
4 after removing mean by stations= 524.156109745


In [9]:
sc.stop()